In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
import pmdarima as pm
import concurrent.futures
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import threading
import re
from datetime import datetime

In [4]:
warnings.filterwarnings("ignore", message="A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.")
warnings.filterwarnings("ignore", message="No supported index is available. Prediction results will be given with an integer index beginning at `start`.")
warnings.filterwarnings("ignore", message="No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.")
warnings.filterwarnings("ignore", message="Non-invertible starting MA parameters found. Using zeros as starting parameters.")
warnings.filterwarnings("ignore", message="Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.")

In [48]:
output_path='./result/'
file_path='./Clean_tapes/Clean_UoB_Set01_2025-01-07tapes.csv'
file_name='Clean_UoB_Set01_2025-01-07tapes.csv'
data = pd.read_csv(file_path)
data_copy = data[['Time','Weighted_Price']].copy()
data_copy['Time'] = pd.to_datetime(data_copy['Time'], unit='s')
match = re.search(r'(\d{4}-\d{2}-\d{2})', file_name)
if match:
    date_str = match.group(1)  # Extracted date string

    # Convert extracted date string to datetime object
    start_time = datetime.strptime(date_str, "%Y-%m-%d")

days_difference = (datetime.strptime(date_str, "%Y-%m-%d") - datetime(1970, 1, 1)).days

data_copy['Time'] = data_copy['Time'].apply(lambda x: x + timedelta(days=days_difference))

data_copy.set_index('Time', inplace=True)

pred = pd.DataFrame([])

In [49]:
data_copy

,Weighted_Price
Time,
2025-01-07 00:00:06.417,336.00
2025-01-07 00:00:11.718,337.00
2025-01-07 00:00:15.469,337.00
2025-01-07 00:00:18.321,338.00
2025-01-07 00:00:19.530,336.20
...,...
2025-01-07 08:29:49.653,368.80
2025-01-07 08:29:51.327,367.60
2025-01-07 08:29:52.412,370.00


In [50]:
# Arima 5s multithreading version

def custom_find_nearest(time_index, target_time):

    # Compute time difference
    time_deltas = abs(time_index - target_time)
    # Find the index with the smallest time difference
    closest_index = time_deltas.argmin()
    return closest_index

def process_data(i):
    current_start = start_time + timedelta(seconds=i*10)
    current_end = current_start + timedelta(seconds=600)
    print(f"{current_start} - {current_end}"+" start")
    print("threads counts:", threading.active_count())
    data_copy_slice = data_copy.loc[current_start:current_end]
    print(data_copy_slice)
    data_copy_slice2 = data_copy.loc[current_end:current_end+timedelta(seconds=10)]
    if data_copy_slice2.index.empty:
        data_copy_slice2 = data_copy.loc[current_end:current_end+timedelta(seconds=15)]
    print(data_copy_slice2)
        

    volatility = np.std(data_copy_slice['Weighted_Price'].pct_change())
    
    multiplier = 1
    
    threshold = volatility * multiplier
    
    print(threshold)

    model = ARIMA(data_copy_slice, order=(1, 1, 1))
    fitted_model = model.fit()
    forecast = fitted_model.forecast(steps=10)
    forecast_values = forecast.values 
    forecast_index = [current_end + timedelta(seconds=x*1) for x in range(1, 11)]  # Create a new time index
    forecast_df = pd.DataFrame(forecast_values, index=forecast_index, columns=['Forecasted_Price'])
    print(f"{current_start} - {current_end}"+" end")
    
    max_price = forecast_df['Forecasted_Price'].max()
    min_price = forecast_df['Forecasted_Price'].min()

    max_time = forecast_df['Forecasted_Price'].idxmax()
    min_time = forecast_df['Forecasted_Price'].idxmin()
    
    max_time = pd.to_datetime(max_time)
    min_time = pd.to_datetime(min_time)
    
    closest_max_time_index = custom_find_nearest(data_copy_slice2.index, max_time)
    closest_min_time_index = custom_find_nearest(data_copy_slice2.index, min_time)
    
    closest_max_time_price = data_copy_slice2.iloc[closest_max_time_index]['Weighted_Price']
    closest_min_time_price = data_copy_slice2.iloc[closest_min_time_index]['Weighted_Price']


    # Compare the time when the highest point and the lowest point occur and return the decision value
    decision = 1 if min_time < max_time else 2
    
    
    act_buy = False if min_price <  (1-threshold)*closest_min_time_price  else True
    
    act_sell = False if max_price >  (1+threshold)*closest_max_time_price else True
    
    
    result = pd.DataFrame({
        'Highest_Price': max_price,
        'Lowest_Price': min_price,
        'Threshold':threshold ,
        'Decision': decision,
        'act_buy':act_buy,
        'act_sell':act_sell
    },  index=[current_end]) 

#     print(result)
    return result

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     futures = [executor.submit(process_data, i) for i in range(3000)]
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(process_data, i) for i in range(3000)]
    

    for future in concurrent.futures.as_completed(futures):
        pred = pd.concat([pred, future.result()])
        
sorted_pred = pred.sort_index()        

2025-01-07 00:00:00 - 2025-01-07 00:10:00 start2025-01-07 00:00:10 - 2025-01-07 00:10:10 start
threads counts: 10

threads counts: 11
2025-01-07 00:00:20 - 2025-01-07 00:10:20 start
threads counts: 11
                         Weighted_Price
Time                                   
2025-01-07 00:00:20.119           336.0
2025-01-07 00:00:20.863           336.0
2025-01-07 00:00:21.948           336.0
2025-01-07 00:00:23.281           333.0
2025-01-07 00:00:23.436           334.0
...                                 ...
2025-01-07 00:10:08.592           337.0
2025-01-07 00:10:09.646           337.0
2025-01-07 00:10:17.148           336.0
2025-01-07 00:10:17.489           336.0
2025-01-07 00:10:17.985           336.0

[353 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 00:10:23.317      335.000000
2025-01-07 00:10:27.192      335.666667
2025-01-07 00:10:29.889      336.000000
0.009244786844533974
                         Weighted_

2025-01-07 00:04:40 - 2025-01-07 00:14:40 start
threads counts: 37
                         Weighted_Price
Time                                   
2025-01-07 00:04:40.178           338.0
2025-01-07 00:04:43.774           337.4
2025-01-07 00:04:47.029           335.0
2025-01-07 00:04:47.370           338.0
2025-01-07 00:04:52.950           339.8
...                                 ...
2025-01-07 00:14:28.124           334.0
2025-01-07 00:14:28.341           334.0
2025-01-07 00:14:29.643           332.5
2025-01-07 00:14:32.960           339.0
2025-01-07 00:14:36.959           332.0

[325 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 00:14:42.694      327.000000
2025-01-07 00:14:43.438      327.000000
2025-01-07 00:14:44.213      328.000000
2025-01-07 00:14:44.492      329.000000
2025-01-07 00:14:45.112      336.333333
2025-01-07 00:14:47.654      329.000000
2025-01-07 00:14:48.212      329.000000
0.008675624379430962
2025-01-

2025-01-07 00:06:10 - 2025-01-07 00:16:10 start2025-01-07 00:06:20 - 2025-01-07 00:16:20 start
threads counts: 47
                         Weighted_Price
Time                                   
2025-01-07 00:06:20.370           337.4
2025-01-07 00:06:24.059           337.0
2025-01-07 00:06:26.570           335.0
2025-01-07 00:06:30.662           339.0
2025-01-07 00:06:31.344           340.0
...                                 ...
2025-01-07 00:16:12.253           335.0
2025-01-07 00:16:16.035           331.0
2025-01-07 00:16:16.872           332.0
2025-01-07 00:16:17.027           331.0
2025-01-07 00:16:17.802           331.0

[336 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 00:16:20.344          331.00
2025-01-07 00:16:20.437          333.00
2025-01-07 00:16:22.142          333.00
2025-01-07 00:16:22.948          330.25
2025-01-07 00:16:26.017          328.00
2025-01-07 00:16:26.761          320.00
0.009483357820552252



2025-01-07 00:08:10 - 2025-01-07 00:18:10 start2025-01-07 00:08:20 - 2025-01-07 00:18:20 start
threads counts: 59
                         Weighted_Price
Time                                   
2025-01-07 00:08:20.805           333.0
2025-01-07 00:08:21.642           332.0
2025-01-07 00:08:21.797           330.0
2025-01-07 00:08:22.572           330.0
2025-01-07 00:08:22.789           330.0
...                                 ...
2025-01-07 00:18:07.263           327.0
2025-01-07 00:18:08.720           327.0
2025-01-07 00:18:15.013           320.0
2025-01-07 00:18:16.129           323.0
2025-01-07 00:18:18.578           323.0

[343 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 00:18:20.004           329.0
2025-01-07 00:18:20.965           330.0
2025-01-07 00:18:24.437           325.6
2025-01-07 00:18:28.002           330.0
2025-01-07 00:18:28.126           330.0

threads counts: 60
0.010747943296384121
                     

2025-01-07 00:10:00 - 2025-01-07 00:20:00 start0.010832050451381584

threads counts: 69
                         Weighted_Price
Time                                   
2025-01-07 00:10:00.129      336.000000
2025-01-07 00:10:00.811      334.000000
2025-01-07 00:10:04.190      337.000000
2025-01-07 00:10:05.430      339.000000
2025-01-07 00:10:05.461      339.000000
...                                 ...
2025-01-07 00:19:54.492      329.000000
2025-01-07 00:19:54.988      328.000000
2025-01-07 00:19:55.980      329.666667
2025-01-07 00:19:57.251      331.000000
2025-01-07 00:19:58.615      332.333333

[351 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 00:20:01.002           320.0
2025-01-07 00:20:01.250           328.0
2025-01-07 00:20:09.403           322.0
0.011081610364888828
2025-01-07 00:10:10 - 2025-01-07 00:20:10 start
threads counts: 70
                         Weighted_Price
Time                                   


...

2025-01-07 07:58:30 - 2025-01-07 08:08:30 end2025-01-07 07:57:20 - 2025-01-07 08:07:20 end
2025-01-07 08:10:10 - 2025-01-07 08:20:10 start
threads counts: 108

2025-01-07 08:10:20 - 2025-01-07 08:20:20 start
threads counts: 108
                         Weighted_Price
Time                                   
2025-01-07 08:10:13.234           359.0
2025-01-07 08:10:13.699           361.0
2025-01-07 08:10:18.597           360.0
2025-01-07 08:10:19.403           360.0
2025-01-07 08:10:19.837           360.0
...                                 ...
2025-01-07 08:20:01.676           376.0
2025-01-07 08:20:05.024           370.8
2025-01-07 08:20:07.597           370.0
2025-01-07 08:20:07.690           370.0
2025-01-07 08:20:09.736           371.0

[328 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 08:20:11.069           374.0
2025-01-07 08:20:14.944           376.0
2025-01-07 08:20:16.804           375.0
2025-01-07 08:20:18.540     

                         Weighted_Price
Time                                   
2025-01-07 08:13:00.696      374.666667
2025-01-07 08:13:09.903      377.000000
2025-01-07 08:13:10.120      376.000000
2025-01-07 08:13:10.430      376.000000
2025-01-07 08:13:12.414      373.000000
...                                 ...
2025-01-07 08:22:49.324      376.000000
2025-01-07 08:22:50.037      376.000000
2025-01-07 08:22:54.625      374.000000
2025-01-07 08:22:55.648      374.000000
2025-01-07 08:22:57.663      375.000000

[307 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 08:23:00.019          371.25
2025-01-07 08:23:00.236          371.00
2025-01-07 08:23:02.995          371.00
2025-01-07 08:23:04.235          370.50
2025-01-07 08:23:05.971          369.00
2025-01-07 08:23:06.808          371.00
2025-01-07 08:23:08.389          367.00
0.00811925495911812
2025-01-07 07:58:20 - 2025-01-07 08:08:20 end
2025-01-07 08:13:10 - 2025-01-

2025-01-07 08:03:00 - 2025-01-07 08:13:00 end
2025-01-07 08:15:40 - 2025-01-07 08:25:40 start
threads counts: 108
                         Weighted_Price
Time                                   
2025-01-07 08:15:41.772           378.4
2025-01-07 08:15:43.167           372.5
2025-01-07 08:15:43.911           362.8
2025-01-07 08:15:44.841           369.6
2025-01-07 08:15:46.360           374.0
...                                 ...
2025-01-07 08:25:28.571           369.0
2025-01-07 08:25:30.555           377.0
2025-01-07 08:25:30.710           374.0
2025-01-07 08:25:32.043           374.0
2025-01-07 08:25:34.120           374.0

[308 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 08:25:41.777      375.000000
2025-01-07 08:25:41.901      375.000000
2025-01-07 08:25:42.304      375.000000
2025-01-07 08:25:43.203      375.000000
2025-01-07 08:25:43.265      371.000000
2025-01-07 08:25:48.504      372.333333
0.007624604897072925
2

2025-01-07 08:03:30 - 2025-01-07 08:13:30 end2025-01-07 08:03:10 - 2025-01-07 08:13:10 end
2025-01-07 08:17:40 - 2025-01-07 08:27:40 start
threads counts: 108

2025-01-07 08:17:50 - 2025-01-07 08:27:50 start
threads counts: 108
                         Weighted_Price
Time                                   
2025-01-07 08:17:50.267           376.0
2025-01-07 08:17:54.700           377.0
2025-01-07 08:18:01.985           377.0
2025-01-07 08:18:02.574           376.0
2025-01-07 08:18:02.729           376.0
...                                 ...
2025-01-07 08:27:39.949           368.0
2025-01-07 08:27:40.476           368.0
2025-01-07 08:27:42.708           371.0
2025-01-07 08:27:44.258           371.0
2025-01-07 08:27:48.939           368.0

[318 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 08:27:51.140           371.0
2025-01-07 08:27:52.132           373.0
2025-01-07 08:27:53.713           373.0
2025-01-07 08:27:54.519     

2025-01-07 08:01:50 - 2025-01-07 08:11:50 end
2025-01-07 08:19:20 - 2025-01-07 08:29:20 start
threads counts: 108
                         Weighted_Price
Time                                   
2025-01-07 08:19:21.221           374.0
2025-01-07 08:19:23.608           374.0
2025-01-07 08:19:28.289           374.0
2025-01-07 08:19:29.033           375.0
2025-01-07 08:19:30.025           375.0
...                                 ...
2025-01-07 08:29:14.933           373.0
2025-01-07 08:29:16.731           373.0
2025-01-07 08:29:17.196           372.0
2025-01-07 08:29:17.971           370.5
2025-01-07 08:29:19.707           368.0

[333 rows x 1 columns]
                         Weighted_Price
Time                                   
2025-01-07 08:29:20.482      367.400000
2025-01-07 08:29:24.574      372.000000
2025-01-07 08:29:26.403      369.000000
2025-01-07 08:29:26.775      367.000000
2025-01-07 08:29:29.658      359.333333
0.00798766068757115
2025-01-07 08:06:40 - 2025-01-07 08:16:40 

In [52]:
initial_capital = 10000
transaction_num = 1
hold_stocks = False
transaction_result = []
stock_num = 0


for index, row in sorted_pred.iterrows():
    operation = row['Decision']
    highest_price = row['Highest_Price']
    lowest_price = row['Lowest_Price']
    act_buy = row['act_buy']
    act_sell = row['act_sell']
    
    if highest_price-lowest_price >= 1 and operation == 1:
        
        if  act_buy == True and act_sell == True  and initial_capital >= highest_price:

            initial_capital -= transaction_num * lowest_price
            stock_num += transaction_num
            initial_capital += stock_num*highest_price
            sell_num = stock_num
            stock_num = 0
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'buy first and sell all', 'buy price':lowest_price,'sell price':highest_price ,'sell number':sell_num ,'buy_num':transaction_num})

        if  act_buy == False and act_sell == True   and hold_stocks == True :

            initial_capital += stock_num*highest_price
            sell_num = stock_num
            stock_num = 0
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'buy faild only sell', 'buy price': 0,'sell price': highest_price,'sell number':sell_num ,'buy_num': 0 })

        if  act_buy == False and act_sell == True  and initial_capital >= highest_price :
            initial_capital -= transaction_num * lowest_price
            stock_num += transaction_num
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'sell faild only buy', 'buy price':lowest_price ,'sell price': 0,'sell number':0 ,'buy_num': transaction_num })

        if  act_buy == False and act_sell == False:
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'sell faild and buy faild', 'buy price':0 ,'sell price': 0,'sell number':0 ,'buy_num': 0 })

if hold_stocks == True:
    final_price = data_copy['Weighted_Price'].iloc[-1]
    initial_capital += final_price*stock_num
    sell_num = stock_num
    stock_num = 0
    hold_stocks = False
    transaction_result.append({'operation': 'sell all', 'buy price':final_price ,'sell price': 0,'sell number':sell_num ,'buy_num': stock_num })
    
transaction_result_df = pd.DataFrame(transaction_result)
print(transaction_result_df)

# Calculate final capital and profit
final_capital = initial_capital
profit = final_capital - 10000
profitability = profit/10000
profitability_percent = "{:.4f}%".format(profitability * 100)
print(f'capital at closing：{final_capital}')
print(f'total_profit：{profit}')
print(f'Profitability：{profitability_percent}')

# profitability_data = {
#     'Final Capital': final_capital,
#     'Profit': profit,
#     'Profitability': profitability,
#     'Profitability Percent': profitability_percent
# }

#     # Convert the dictionary to a DataFrame
# profitability_df = pd.DataFrame([profitability_data])

    
# if match:
#     date_string = match.group(0)
#     new_file_name = f'{date_string}_transaction_result.csv'
#     new_file_name1 = f'{date_string}_profitability.csv'
#     transaction_result_df.to_csv(os.path.join(output_folder, new_file_name))
#     profitability_df.to_csv(os.path.join(output_folder, new_file_name1))


                    operation   buy price  sell price  sell number  buy_num
0      buy first and sell all  330.572525  334.199997            1        1
1      buy first and sell all  331.788200  333.654912            1        1
2      buy first and sell all  324.864504  331.297835            1        1
3      buy first and sell all  332.435346  333.738823            1        1
4      buy first and sell all  332.327612  333.755220            1        1
..                        ...         ...         ...          ...      ...
608    buy first and sell all  360.932881  365.957118            2        1
609    buy first and sell all  364.608320  366.249130            1        1
610  sell faild and buy faild    0.000000    0.000000            0        0
611       sell faild only buy  358.269940    0.000000            0        1
612                  sell all  371.250000    0.000000            1        0

[613 rows x 5 columns]
capital at closing：11637.039467244693
total_profit：1637.03946724